In [3]:
import pandas as pd 
import numpy as np
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.sandbox.stats.multicomp import multipletests 

In [42]:
data = pd.read_csv('/content/_cc291fb54b232482695c784776896188_gene_high_throughput_sequencing (1).csv')

In [33]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,C1orf159,LOC254099,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,FAM132A,UBE2J2,SCNN1D,ACAP3,PUSL1,CPSF3L,GLTPD1,DVL1,MXRA8,AURKAIP1,CCNL2,LOC148413,MRPL20,LOC441869,VWA1,ATAD3C,ATAD3B,ATAD3A,C1orf70,SSU72,LOC643988,...,RENBP,HCFC1,TMEM187,IRAK1,MECP2,TKTL1,FLNA,EMD,RPL10,SNORA70,DNASE1L1,TAZ,ATP6AP1,GDI1,FAM50A,PLXNA3,LAGE3,UBL4A,SLC10A3,FAM3A,G6PD,GAB3,DKC1,MPP1,F8,FUNDC2,MTCP1NB,MTCP1,VBP1,RAB39B,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,57.553312,42.455733,6.614939,7.786382,14.283751,18.752091,65.825821,17.471334,14.135344,42.949564,35.946652,50.945017,19.738267,52.172044,34.753563,27.810055,83.303689,59.428091,42.306459,13.209938,20.161089,1.257614,66.621905,3.772842,40.210889,30.278655,14.576032,73.168939,34.692832,...,3.164926,20.369209,5.577156,22.438224,20.981186,1.257614,146.983123,33.074695,4.761250,5.185276,15.282298,24.761483,66.017756,51.888333,40.000566,19.950798,19.306187,26.007591,10.139210,40.105866,43.047651,4.761250,30.624906,9.714340,5.577156,4.761250,10.139210,2.408148,12.214621,3.164926,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,119.572567,47.367114,4.567931,8.049497,31.421187,19.744990,123.120249,20.615707,16.865244,73.826942,45.474497,63.043023,28.021783,62.483096,60.409871,41.747570,124.715481,128.770634,33.452597,17.876777,28.947097,1.815112,113.503762,1.815112,45.954959,36.946925,15.222429,127.777507,66.567847,...,12.355210,22.255175,9.075559,38.347144,21.245338,1.815112,152.191825,49.097571,8.577883,1.815112,23.965891,30.426817,160.727095,82.216081,60.045184,40.681726,70.040738,44.200859,13.379397,60.045184,72.566645,1.815112,21.451109,6.199787,1.815112,6.199787,5.445335,3.475674,9.075559,1.815112,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,55.579752,41.741942,3.978294,6.232790,24.317644,28.662801,64.494789,14.543176,12.407734,37.531155,32.074035,63.596176,13.303109,42.016789,26.142516,22.853562,76.118706,56.095106,40.053273,13.303109,20.461953,2.077597,58.702060,2.077597,42.153541,30.416278,16.750119,70.137476,35.481715,...,5.228503,20.178729,3.978294,20.741310,22.085154,3.978294,145.308037,27.008755,6.232790,7.096343,16.750119,24.553171,58.702060,46.192467,44.154018,20.178729,13.728915,23.839609,7.865653,37.684188,39.474334,5.228503,38.139602,9.213541,6.232790,2.077597,12.407734,7.096343,10.387983,6.232790,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,54.454714,35.933084,2.066576,7.823932,24.422935,15.963105,54.245171,10.870009,14.854454,23.227927,27.905160,49.869762,8.520713,31.903907,16.999606,20.631293,34.477366,39.983492,32.784180,10.870009,24.884876,2.066576,53.398778,9.164670,40.125656,21.968009,14.066895,39.983492,28.907458,...,2.066576,13.656094,7.058702,16.661272,7.823932,2.066576,90.135274,17.976444,2.066576,9.164670,13.656094,15.602310,53.611629,24.654987,35.933084,3.957193,6.199729,11.871578,9.164670,32.434938,25.112660,2.066576,5.200770,14.466035,7.823932,7.823932,17.656837,2.066576,17.331337,5.200770,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,48.134682,46.507417,7.179340,8.151497,23.096282,29.007823,78.309425,16.586217,18.698195,42.135928,39.393867,65.111910,15.819691,60.974762,27.872385,38.0466

##Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения).

## 1 Задание
Применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)

In [46]:
s = []
folds1 = []
p = []
folds2 = []
def fold_change(t, c):
  if t >= c:
    return t/c
  else:
    return -c/t
for i in list(data.columns[2:]):
  normal = data[data.Diagnosis == 'normal'][i]
  cancer = data[data.Diagnosis == 'cancer'][i]
  early = data[data.Diagnosis == 'early neoplasia'][i]
  control_1_mean, treatment_1_mean = np.mean(data[data.Diagnosis == 'normal'][i]), np.mean(data[data.Diagnosis == 'early neoplasia'][i])
  control_2_mean, treatment_2_mean = np.mean(data[data.Diagnosis == 'early neoplasia'][i]), np.mean(data[data.Diagnosis == 'cancer'][i])
  s.append(stats.ttest_ind(normal, early,  equal_var=False)[1])
  folds1.append(abs(fold_change(treatment_1_mean, control_1_mean)))
  p.append(stats.ttest_ind(cancer, early,  equal_var=False)[1])
  folds2.append(abs(fold_change(treatment_2_mean, control_2_mean)))


In [49]:
susy = 0
pus = 0
for i in range(len(s)):
  if s[i] < 0.05:
    susy+=1
for j in range(len(p)):
  if p[j] < 0.05:
    pus+=1
print(susy)
print(pus)
print(susy+pus)

1575
3490
5065


Вот такое большое получилось количество статистически значимых отличий. Оно и понятно - это специфика множественной проверки гипотез. Проведем попровку.

## 2 Задание
применим поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части

Выведем количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно вывести с учетом практической значимости: посчитаем для каждого значимого изменения fold change и выпишем в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.5.

In [56]:
reject, p_corrected, a1, a2 = multipletests(s, 
                                            alpha = 0.05/2, 
                                            method = 'holm')
num1 = 0
for i in range(len(p_corrected)):
  if p_corrected[i] < 0.05:
    if folds1[i] > 1.5:
      num1 += 1
num1

4

In [57]:
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05/2, 
                                            method = 'holm')
num2 = 0
for i in range(len(p_corrected)):
  if p_corrected[i] < 0.05:
    if folds2[i] > 1.5:
      num2 += 1
num2

91

при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

## 3 Задание
То же самое, но поправка Бенджамини-Хохберга

In [58]:
reject, p_corrected, a1, a2 = multipletests(s, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh')
num1 = 0
for i in range(len(p_corrected)):
  if p_corrected[i] < 0.05:
    if folds1[i] > 1.5:
      num1 += 1
num1

23

In [61]:
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh')
num2 = 0
for i in range(len(p_corrected)):
  if p_corrected[i] < 0.05:
    if folds2[i] > 1.5:
      num2 += 1
num2

670

 Обратите внимание, что методы коррекции, которые контролируют FDR(Бенджамини-Хохберга), допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER(Холма). Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0, когда они есть, и будут чаще отклонять H_0, когда отличий нет).